<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

Для начала загрузими и изучим исходные данные:

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv('/datasets/Churn.csv')
data.head(15)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Итак, в исходном датафрейме находятся 10000 сторок, каждая из которых соответствует опредленному клиенту "Бета-банка", и 14 столбцов. Три столбца имеют категориальный тип, также в одном из столбцов наблюдаются пропуски.

In [ ]:
data.duplicated().sum()

0

In [ ]:
data['CustomerId'].duplicated().sum()

0

Явных дубликатов в датафрейме нет, также как и нет повторяющихся значений в столбце 'CustomerId', а значит, каждому клиенту банка соответствует только одна строка данных.

Попробуем избавится от пропусков в столбце 'Tenure'.

Так как явной зависимости этих пропусков от других данных не наблюдается, а заменять их средним или медианным значением не представляется необходимым, было принято решение удалить строки с пропущенными значениями:

In [ ]:
data = data.dropna().reset_index(drop=True)

<div class='alert alert-success'> ✔️Ок, еще бы рекомендовал явно вывести количество пропусков, чтобы показать, что остается данных достаточно.

</div>

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9091 entries, 0 to 9090
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        9091 non-null   int64  
 1   CustomerId       9091 non-null   int64  
 2   Surname          9091 non-null   object 
 3   CreditScore      9091 non-null   int64  
 4   Geography        9091 non-null   object 
 5   Gender           9091 non-null   object 
 6   Age              9091 non-null   int64  
 7   Tenure           9091 non-null   float64
 8   Balance          9091 non-null   float64
 9   NumOfProducts    9091 non-null   int64  
 10  HasCrCard        9091 non-null   int64  
 11  IsActiveMember   9091 non-null   int64  
 12  EstimatedSalary  9091 non-null   float64
 13  Exited           9091 non-null   int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 994.5+ KB


Для дальнейшего исследования и обучения моделей удалим столбцы, которые не имеют влияния на наш результативный признак (Exited):

In [ ]:
data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

Преобразуем категориальные признаки в количественные с помощью метода прямого кодирования (One-Hot Encoding, OHE), избегая дамми-ловушки:

In [ ]:
data = pd.get_dummies(data, drop_first=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9091 entries, 0 to 9090
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        9091 non-null   int64  
 1   Age                9091 non-null   int64  
 2   Tenure             9091 non-null   float64
 3   Balance            9091 non-null   float64
 4   NumOfProducts      9091 non-null   int64  
 5   HasCrCard          9091 non-null   int64  
 6   IsActiveMember     9091 non-null   int64  
 7   EstimatedSalary    9091 non-null   float64
 8   Exited             9091 non-null   int64  
 9   Geography_Germany  9091 non-null   uint8  
 10  Geography_Spain    9091 non-null   uint8  
 11  Gender_Male        9091 non-null   uint8  
dtypes: float64(3), int64(6), uint8(3)
memory usage: 666.0 KB


In [ ]:
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


Поделим наши данные на обучающую, валидационную и тестовую выборки:

In [ ]:
features = data.drop('Exited', axis=1)
target = data['Exited']

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.25, random_state=12345)


## Исследование задачи

Для начала исследуем баланс классов:

In [ ]:
data['Exited'].value_counts(normalize=True)

0    0.796062
1    0.203938
Name: Exited, dtype: float64

Видим, что клиенты, ушедшие из банка, составляют около 20% от общего числа клиентов, соответственно, наблюдается дисбаланс классов.

Попробуем обучить модель без учета дисбаланса и изучим полученные результаты:

Так как перед нами данные с категориальным целевым признаком (факт ухода клиента из банка (0 или 1)), то решаем задачу классификации (даже бинарной (двоичной) классификации). Нами было изучено 3 модели, подходящии для данной задачи - дерево решений, случайный лес и логистическая регрессия. Исследуем качество каждой из модели на валидационной выборке, меняя гиперпараметры:

In [ ]:
best_model = None
best_result = 0
best_depth = 0
best_split = 0
best_leaf = 0

for depth in range(1, 10):
    for split in range(2, 10):
        for leaf in range(1, 10):
            model = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_split=split, min_samples_leaf=leaf)
            model.fit(features_train, target_train)
            predictions_valid = model.predict(features_valid)
            result = f1_score(target_valid, predictions_valid)
            if result > best_result:
                best_model = model
                best_result = result
                best_depth = depth
                best_split = split
                best_leaf = leaf


print('F1 лучшей модели:', best_result)
print('Глубина дерева:', best_depth)
print('Минимальное количество примеров для разделения:', best_split)
print('Минимальное количество объектов в листе:', best_leaf)

probabilities_valid = best_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC:', auc_roc)

F1 лучшей модели: 0.562596599690881
Глубина дерева: 9
Минимальное количество примеров для разделения: 3
Минимальное количество объектов в листе: 1
Значение AUC-ROC: 0.8091721988890395


In [ ]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 71, 10):
    for depth in range (1, 21):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = f1_score(target_valid, predictions_valid)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth

print('F1 лучшей модели:', best_result)
print('Количество деревьев:', best_est)
print('Максимальная глубина:', best_depth)

probabilities_valid = best_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC:', auc_roc)

F1 лучшей модели: 0.5616438356164384
Количество деревьев: 40
Максимальная глубина: 20
Значение AUC-ROC: 0.8203100302578693


In [ ]:
best_model = None
best_score = 0
best_iter = 0

for iter in range(100, 1500, 10):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=iter)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    score = f1_score(target_valid, predictions_valid)
    if score > best_score:
        best_model = model
        best_score = score
        best_iter = iter

print('F1 лучшей модели:', best_score)
print('Максимальное количество итераций обучения:', best_iter)

probabilities_valid = best_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC:', auc_roc)

F1 лучшей модели: 0.0681265206812652
Максимальное количество итераций обучения: 100
Значение AUC-ROC: 0.6734483433440214


Без учета дисбаланса классов ни одна из изученных моделей (дерево решений, случайный лес, логистическая регрессия) не проходит заданный порог для F1-меры в 0,59.

Лучшее значение F1-меры у дерева решений - 0,563, у случайного леса - 0,562, а у логистической регрессия и вовсе 0,068.

## Борьба с дисбалансом

Попробуем улучшить качество моделей, учитывая дисбаланс классов. Нами было изучено 3 способа борьбы с дисбалансом: взвешивание классов (class_weight='balanced'), увеличение выборки (upsampling) и уменьшение выборки (downsampling). Проверим значения F1-меры используя каждый из методов:

Взвешивание классов:

In [ ]:
best_model = None
best_result = 0
best_depth = 0
best_split = 0
best_leaf = 0

for depth in range(1, 10):
    for split in range(2, 10):
        for leaf in range(1, 10):
            model = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_split=split, min_samples_leaf=leaf, class_weight='balanced')
            model.fit(features_train, target_train)
            predictions_valid = model.predict(features_valid)
            result = f1_score(target_valid, predictions_valid)
            if result > best_result:
                best_model = model
                best_result = result
                best_depth = depth
                best_split = split
                best_leaf = leaf

print('F1 лучшей модели:', best_result)
print('Глубина дерева:', best_depth)
print('Минимальное количество примеров для разделения:', best_split)
print('Минимальное количество объектов в листе:', best_leaf)

probabilities_valid = best_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC:', auc_roc)

F1 лучшей модели: 0.5654345654345655
Глубина дерева: 7
Минимальное количество примеров для разделения: 2
Минимальное количество объектов в листе: 7
Значение AUC-ROC: 0.831164664529046


In [ ]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 100, 10):
    for depth in range (1, 21):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, class_weight='balanced')
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = f1_score(target_valid, predictions_valid)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth

print('F1 лучшей модели:', best_result)
print('Количество деревьев:', best_est)
print('Максимальная глубина:', best_depth)

probabilities_valid = best_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC:', auc_roc)

F1 лучшей модели: 0.6112676056338029
Количество деревьев: 90
Максимальная глубина: 10
Значение AUC-ROC: 0.8459972301253971


In [ ]:
best_model = None
best_score = 0
best_iter = 0

for iter in range(100, 1500, 10):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=iter, class_weight='balanced')
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    score = f1_score(target_valid, predictions_valid)
    if score > best_score:
        best_model = model
        best_score = score
        best_iter = iter

print('F1 лучшей модели:', best_score)
print('Максимальное количество итераций обучения:', best_iter)

probabilities_valid = best_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC:', auc_roc)

F1 лучшей модели: 0.34167893961708395
Максимальное количество итераций обучения: 100
Значение AUC-ROC: 0.570787984163543


Увеличение выборки (upsampling):

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)

    return features_upsampled, target_upsampled

<li>   <font color='steelblue'>
    Найдем отношение количества клиентов, которые остались в банке, к тем, которые ушли (0 к 1). Это число будет количеством повторений при увеличении выборки.
</font>

In [ ]:
len(data.loc[data['Exited'] == 0])/len(data.loc[data['Exited'] == 1])

3.9034519956850056

Округлим до 4

In [ ]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

model = DecisionTreeClassifier(random_state=12345, max_depth=7, min_samples_split=2, min_samples_leaf=7)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC:', auc_roc)

F1: 0.5552238805970149
Значение AUC-ROC: 0.8258667148383987


In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=90, max_depth=10)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC:', auc_roc)

F1: 0.5974999999999999
Значение AUC-ROC: 0.8490757048879254


In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear', max_iter=100)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC:', auc_roc)

F1: 0.4493392070484581
Значение AUC-ROC: 0.7138881362055729


Уменьшение выборки (downsampling):

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)

    return features_downsampled, target_downsampled

Найдем долю положительных и отрицательных ответов:

In [ ]:
data['Exited'].value_counts(normalize=True)

0    0.796062
1    0.203938
Name: Exited, dtype: float64

In [ ]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.2)

model = DecisionTreeClassifier(random_state=12345, max_depth=7, min_samples_split=2, min_samples_leaf=7)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC:', auc_roc)

F1: 0.5099818511796733
Значение AUC-ROC: 0.8007863658944135


In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=90, max_depth=10)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC:', auc_roc)

F1: 0.5666337611056269
Значение AUC-ROC: 0.8494934441283175


In [ ]:
model = LogisticRegression(solver='liblinear', random_state=12345, max_iter=100)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC:', auc_roc)

F1: 0.4199706314243759
Значение AUC-ROC: 0.7117204082554307


Наилучшее качество показала модель случайного леса с использованием метода взвешивания классов с F1 = 0,61, что проходит заданный порог в 0,59.

Дополнительно измерим AUC-ROC и сравним ее значение с F1-мерой:

AUC-ROC (площадь под ROC-кривой):

In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=90, max_depth=10, class_weight='balanced')
model.fit(features_train, target_train)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

roc_auc_score(target_valid, probabilities_one_valid)

0.8459972301253971

Метрика AUC-ROC значительно больше, чем у случайной модели (0,5) и F1-меры, однако нельзя сказать, что близка к идеальной AUC-ROC = 1.

## Тестирование модели

Проведен финальное тестирование лучшей модели (случайный лес с использованием техники взвешивания классов) на тестовой выборке:

In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=90, max_depth=10, class_weight='balanced')
model.fit(features_train, target_train)
pred_test = model.predict(features_test)
f1_score(target_test, pred_test)

0.6013793103448275

С заданным порогом F1-меры в 0,59 модель проходит проверку на качество с результатом 0,60.